In [26]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import openpyxl as op

# Reading in Data Sets

In [210]:
coffee_df = pd.read_excel("Monthly Return_Coffee Products.xlsx", index_col = "Names Date", parse_dates = True)
alc_df = pd.read_excel("Monthly Return_Alcoholic Products.xlsx", index_col = "Names Date", parse_dates = True)
nber_df = pd.read_excel("NBER classification of economic cycle.xlsx", skiprows = 1)
fama_df = pd.read_excel("FamaFrench.xlsx",  index_col = 0, parse_dates = True)

In [211]:
fred_df = pd.read_csv("FRED-NBER-recession-dummies.csv", parse_dates = True)

In [212]:
coffee_df.head()


,PERMNO,Standard Industrial Classification Code,Ticker Symbol,Company Name,Price or Bid/Ask Average
Names Date,,,,,
1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875
1973-01-31,34367,2095,FARM,FARMER BROTHERS CO,-20.500
1973-02-28,34367,2095,FARM,FARMER BROTHERS CO,-17.250
1973-03-30,34367,2095,FARM,FARMER BROTHERS CO,-16.125
1973-04-30,34367,2095,FARM,FARMER BROTHERS CO,-12.875


In [213]:
alc_df.head()

,PERMNO,Standard Industrial Classification Code,Ticker Symbol,Company Name,Price or Bid/Ask Average
Names Date,,,,,
2011-10-31,10225,2082,BEAM,BEAM INC,49.430000
2011-11-30,10225,2082,BEAM,BEAM INC,52.520000
2011-12-30,10225,2082,BEAM,BEAM INC,51.230000
2012-01-31,10225,2082,BEAM,BEAM INC,52.310001
2012-02-29,10225,2082,BEAM,BEAM INC,55.080002


In [34]:
nber_df.head()

,Unnamed: 0,Unnamed: 1,Peak month number,Trough month number,"Duration, peak to trough","Duration, trough to peak","Duration, trough to trough","Duration, peak to peak"
0,NaN,December 1854 (1854Q4),NaN,660.0,NaN,NaN,NaN,NaN
1,June 1857 (1857Q2),December 1858 (1858Q4),690,708.0,18.0,30.0,48.0,NaN
2,October 1860 (1860Q3),June 1861 (1861Q3),730,738.0,8.0,22.0,30.0,40.0
3,April 1865 (1865Q1),December 1867 (1868Q1),784,816.0,32.0,46.0,78.0,54.0
4,June 1869 (1869Q2),December 1870 (1870Q4),834,852.0,18.0,18.0,36.0,50.0


In [31]:
fama_df.head()

,Excess Return on the Market,Small-Minus-Big Return,High-Minus-Low Return,Momentum Factor
Date (SAS). Last Trading Day of the Month,,,,
1926-07-31,0.0296,-0.0238,-0.0273,NaN
1926-08-31,0.0264,-0.0147,0.0414,NaN
1926-09-30,0.0036,-0.0139,0.0012,NaN
1926-10-30,-0.0324,-0.0013,0.0065,NaN
1926-11-30,0.0253,-0.0016,-0.0038,NaN


In [48]:
# Checking shapes of all dfs

alc_df.shape, coffee_df.shape, fama_df.shape, nber_df.shape

((12038, 5), (1119, 5), (1134, 4), (43, 8))

# Splitting CRSP datasets
- I need to reduce the dimensionality such that one date has one long stretch of data?

## Alcohol

In [68]:
grouped_alc = alc_df.groupby("Names Date", dropna = False)

In [71]:
lst = [grouped_alc.get_group(x) for x in grouped_alc.groups]

In [73]:
lst[0]

,PERMNO,Standard Industrial Classification Code,Ticker Symbol,Company Name,Price or Bid/Ask Average
Names Date,,,,,
1962-07-31,19764,2082,PFB,PFEIFFER BREWING CO,2.625
1962-07-31,19967,2082,GBL,GOEBEL BREWING CO,1.250
1962-07-31,22381,2083,RUP,RUPPERT JACOB CO,13.250
1962-07-31,23106,2082,CNB,CANADIAN BREWERIES LTD,8.500
1962-07-31,23683,2082,FAL,FALSTAFF BREWING CORP,32.500
1962-07-31,25339,2082,DRE,DREWRYS LTD USA INC,23.500
1962-07-31,34489,2082,NBW,NATIONAL BREWING CO MICH,-4.125
1962-07-31,13354,2085,DR,NATIONAL DISTILLERS & CHEM CORP,24.875
1962-07-31,17670,2085,ADC,AMERICAN DISTILLING CO,37.000


# Merging on Date Index
- The problem is that CRSP exports are very messy - for a given date column, there can be multiple companies all blended into the same dataframe 
    - Not sure if this is a problem yet

In [214]:
coffee_df.index = pd.to_datetime(coffee_df.index, utc = True)
alc_df.index = pd.to_datetime(alc_df.index, utc = True)


In [225]:
alc_coff = pd.merge(coffee_df, alc_df, how = "inner", right_index = True, left_index = True)

In [216]:
alc_coff.head()

,PERMNO_x,Standard Industrial Classification Code_x,Ticker Symbol_x,Company Name_x,Price or Bid/Ask Average_x,PERMNO_y,Standard Industrial Classification Code_y,Ticker Symbol_y,Company Name_y,Price or Bid/Ask Average_y
Names Date,,,,,,,,,,
1972-12-29 00:00:00+00:00,34367,2095,FARM,FARMER BROTHERS CO,-25.875,19764,2082,ABW,ASSOCIATED BREWING CO,7.125
1972-12-29 00:00:00+00:00,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23106,2082,CNB,CANADIAN BREWERIES LTD,5.375
1972-12-29 00:00:00+00:00,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23683,2082,FAL,FALSTAFF BREWING CORP,5.125
1972-12-29 00:00:00+00:00,34367,2095,FARM,FARMER BROTHERS CO,-25.875,38608,2082,GENBB,GENESEE BREWING INC,-12.500
1972-12-29 00:00:00+00:00,34367,2095,FARM,FARMER BROTHERS CO,-25.875,42323,2082,RG,RHEINGOLD CORP,18.125


In [217]:
alc_coff.shape

(18023, 10)

In [77]:
alc_coff.shape

(18023, 10)

In [78]:
alc_coff.isna().sum()

PERMNO_x                                       0
Standard Industrial Classification Code_x      0
Ticker Symbol_x                                0
Company Name_x                                 0
Price or Bid/Ask Average_x                    34
PERMNO_y                                       0
Standard Industrial Classification Code_y      0
Ticker Symbol_y                               21
Company Name_y                                 0
Price or Bid/Ask Average_y                   144
dtype: int64

In [79]:
train0df = pd.merge(alc_coff, fama_df, how = "inner", left_index = True, right_index = True)

In [80]:
train0df.head()

,PERMNO_x,Standard Industrial Classification Code_x,Ticker Symbol_x,Company Name_x,Price or Bid/Ask Average_x,PERMNO_y,Standard Industrial Classification Code_y,Ticker Symbol_y,Company Name_y,Price or Bid/Ask Average_y,Excess Return on the Market,Small-Minus-Big Return,High-Minus-Low Return,Momentum Factor
1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,19764,2082,ABW,ASSOCIATED BREWING CO,7.125,0.0062,-0.0195,-0.0219,0.0498
1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23106,2082,CNB,CANADIAN BREWERIES LTD,5.375,0.0062,-0.0195,-0.0219,0.0498
1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23683,2082,FAL,FALSTAFF BREWING CORP,5.125,0.0062,-0.0195,-0.0219,0.0498
1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,38608,2082,GENBB,GENESEE BREWING INC,-12.500,0.0062,-0.0195,-0.0219,0.0498
1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,42323,2082,RG,RHEINGOLD CORP,18.125,0.0062,-0.0195,-0.0219,0.0498


In [81]:
train0df.shape

(17711, 14)

In [82]:
train0df.isna().sum()

PERMNO_x                                       0
Standard Industrial Classification Code_x      0
Ticker Symbol_x                                0
Company Name_x                                 0
Price or Bid/Ask Average_x                    34
PERMNO_y                                       0
Standard Industrial Classification Code_y      0
Ticker Symbol_y                               21
Company Name_y                                 0
Price or Bid/Ask Average_y                   144
Excess Return on the Market                    0
Small-Minus-Big Return                         0
High-Minus-Low Return                          0
Momentum Factor                                0
dtype: int64

In [97]:
fred_df.head()

,DATE,USRECQM
0,1854-10-01,1
1,1855-01-01,0
2,1855-04-01,0
3,1855-07-01,0
4,1855-10-01,0


In [98]:
fred_df.shape

(669, 2)

In [109]:
fred_df["Monthly"] = pd.to_datetime(fred_df["DATE"]).dt.to_period("M")
new_fred = fred_df.set_index("Monthly").resample("M").sum()

In [114]:
fred_df.head(20)

,DATE,USRECQM,Monthly
0,1854-10-01,1,1854-10
1,1855-01-01,0,1855-01
2,1855-04-01,0,1855-04
3,1855-07-01,0,1855-07
4,1855-10-01,0,1855-10
5,1856-01-01,0,1856-01
6,1856-04-01,0,1856-04
7,1856-07-01,0,1856-07
8,1856-10-01,0,1856-10
9,1857-01-01,0,1857-01


# I assume that the trend can be forward filled
- That is, the recession trend continues until NBER says otherwise

In [112]:
fred_alt = new_fred.fillna(method = "ffill")

In [117]:
fred_alt.head()

,DATE,USRECQM
Monthly,,
1854-10,1854-10-01,1.0
1854-11,1854-10-01,1.0
1854-12,1854-10-01,1.0
1855-01,1855-01-01,0.0
1855-02,1855-01-01,0.0


In [118]:
fred_alt.shape

(2005, 2)

In [120]:
fred_alt.dtypes

DATE        object
USRECQM    float64
dtype: object

In [139]:
fred_alt.index = fred_alt.index.to_timestamp()

AttributeError: 'DatetimeIndex' object has no attribute 'to_timestamp'

In [140]:
fred_alt = fred_alt.reset_index()

In [147]:
fred_alt = fred_alt.rename(columns = {"Monthly": "index"})

In [155]:
fred_alt.dtypes

index      datetime64[ns]
DATE               object
USRECQM           float64
dtype: object

In [161]:
fred_alt.head()

,index,DATE,USRECQM
0,1854-10-01,1854-10-01,1.0
1,1854-11-01,1854-10-01,1.0
2,1854-12-01,1854-10-01,1.0
3,1855-01-01,1855-01-01,0.0
4,1855-02-01,1855-01-01,0.0


# Merging Everything

In [156]:
train0df.dtypes

level_0                                               int64
index                                        datetime64[ns]
PERMNO_x                                              int64
Standard Industrial Classification Code_x             int64
Ticker Symbol_x                                      object
Company Name_x                                       object
Price or Bid/Ask Average_x                          float64
PERMNO_y                                              int64
Standard Industrial Classification Code_y             int64
Ticker Symbol_y                                      object
Company Name_y                                       object
Price or Bid/Ask Average_y                          float64
Excess Return on the Market                         float64
Small-Minus-Big Return                              float64
High-Minus-Low Return                               float64
Momentum Factor                                     float64
dtype: object

In [138]:
train0df = train0df.reset_index()
train0df.head()

,level_0,index,PERMNO_x,Standard Industrial Classification Code_x,Ticker Symbol_x,Company Name_x,Price or Bid/Ask Average_x,PERMNO_y,Standard Industrial Classification Code_y,Ticker Symbol_y,Company Name_y,Price or Bid/Ask Average_y,Excess Return on the Market,Small-Minus-Big Return,High-Minus-Low Return,Momentum Factor
0,0,1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,19764,2082,ABW,ASSOCIATED BREWING CO,7.125,0.0062,-0.0195,-0.0219,0.0498
1,1,1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23106,2082,CNB,CANADIAN BREWERIES LTD,5.375,0.0062,-0.0195,-0.0219,0.0498
2,2,1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23683,2082,FAL,FALSTAFF BREWING CORP,5.125,0.0062,-0.0195,-0.0219,0.0498
3,3,1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,38608,2082,GENBB,GENESEE BREWING INC,-12.500,0.0062,-0.0195,-0.0219,0.0498
4,4,1972-12-29,34367,2095,FARM,FARMER BROTHERS CO,-25.875,42323,2082,RG,RHEINGOLD CORP,18.125,0.0062,-0.0195,-0.0219,0.0498


In [180]:
train0df["index"] = pd.to_datetime(train0df["index"], utc = True)
fred_alt["index"] = pd.to_datetime(fred_alt["index"], utc = True)

train0df = train0df.set_index("index")
fred_alt = fred_alt.set_index("index")

In [184]:
train0df.head()

,level_0,PERMNO_x,Standard Industrial Classification Code_x,Ticker Symbol_x,Company Name_x,Price or Bid/Ask Average_x,PERMNO_y,Standard Industrial Classification Code_y,Ticker Symbol_y,Company Name_y,Price or Bid/Ask Average_y,Excess Return on the Market,Small-Minus-Big Return,High-Minus-Low Return,Momentum Factor
index,,,,,,,,,,,,,,,
1972-12-29 00:00:00+00:00,0,34367,2095,FARM,FARMER BROTHERS CO,-25.875,19764,2082,ABW,ASSOCIATED BREWING CO,7.125,0.0062,-0.0195,-0.0219,0.0498
1972-12-29 00:00:00+00:00,1,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23106,2082,CNB,CANADIAN BREWERIES LTD,5.375,0.0062,-0.0195,-0.0219,0.0498
1972-12-29 00:00:00+00:00,2,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23683,2082,FAL,FALSTAFF BREWING CORP,5.125,0.0062,-0.0195,-0.0219,0.0498
1972-12-29 00:00:00+00:00,3,34367,2095,FARM,FARMER BROTHERS CO,-25.875,38608,2082,GENBB,GENESEE BREWING INC,-12.500,0.0062,-0.0195,-0.0219,0.0498
1972-12-29 00:00:00+00:00,4,34367,2095,FARM,FARMER BROTHERS CO,-25.875,42323,2082,RG,RHEINGOLD CORP,18.125,0.0062,-0.0195,-0.0219,0.0498


In [201]:
train0df.shape

(17711, 15)

In [183]:
fred_alt.tail()

,DATE,USRECQM
index,,
2021-06-01 00:00:00+00:00,2021-04-01,0.0
2021-07-01 00:00:00+00:00,2021-07-01,0.0
2021-08-01 00:00:00+00:00,2021-07-01,0.0
2021-09-01 00:00:00+00:00,2021-07-01,0.0
2021-10-01 00:00:00+00:00,2021-10-01,0.0


In [197]:
tol = pd.Timedelta(weeks = 4)

main_df = pd.merge_asof(train0df, fred_alt, left_index = True, right_index = True, direction = "nearest", tolerance = tol)

In [198]:
main_df

,level_0,PERMNO_x,Standard Industrial Classification Code_x,Ticker Symbol_x,Company Name_x,Price or Bid/Ask Average_x,PERMNO_y,Standard Industrial Classification Code_y,Ticker Symbol_y,Company Name_y,Price or Bid/Ask Average_y,Excess Return on the Market,Small-Minus-Big Return,High-Minus-Low Return,Momentum Factor,DATE,USRECQM
index,,,,,,,,,,,,,,,,,
1972-12-29 00:00:00+00:00,0,34367,2095,FARM,FARMER BROTHERS CO,-25.875,19764,2082,ABW,ASSOCIATED BREWING CO,7.125000,0.0062,-0.0195,-0.0219,0.0498,1973-01-01,0.0
1972-12-29 00:00:00+00:00,1,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23106,2082,CNB,CANADIAN BREWERIES LTD,5.375000,0.0062,-0.0195,-0.0219,0.0498,1973-01-01,0.0
1972-12-29 00:00:00+00:00,2,34367,2095,FARM,FARMER BROTHERS CO,-25.875,23683,2082,FAL,FALSTAFF BREWING CORP,5.125000,0.0062,-0.0195,-0.0219,0.0498,1973-01-01,0.0
1972-12-29 00:00:00+00:00,3,34367,2095,FARM,FARMER BROTHERS CO,-25.875,38608,2082,GENBB,GENESEE BREWING INC,-12.500000,0.0062,-0.0195,-0.0219,0.0498,1973-01-01,0.0
1972-12-29 00:00:00+00:00,4,34367,2095,FARM,FARMER BROTHERS CO,-25.875,42323,2082,RG,RHEINGOLD CORP,18.125000,0.0062,-0.0195,-0.0219,0.0498,1973-01-01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 00:00:00+00:00,17706,90679,2095,JVA,COFFEE HOLDING CO INC,3.840,29938,2085,BF,BROWN FORMAN CORP,73.470001,0.0463,0.0483,-0.0156,-0.0242,2021-01-01,0.0
2020-12-31 00:00:00+00:00,17707,90679,2095,JVA,COFFEE HOLDING CO INC,3.840,29946,2085,BF,BROWN FORMAN CORP,79.430000,0.0463,0.0483,-0.0156,-0.0242,2021-01-01,0.0
2020-12-31 00:00:00+00:00,17708,90679,2095,JVA,COFFEE HOLDING CO INC,3.840,64899,2084,STZ,CONSTELLATION BRANDS INC,219.330002,0.0463,0.0483,-0.0156,-0.0242,2021-01-01,0.0


In [200]:
main_df.shape

(17711, 17)

In [199]:
main_df["USRECQM"].

0

In [203]:
main_df.loc[main_df["USRECQM"] == 1.0, :]

,level_0,PERMNO_x,Standard Industrial Classification Code_x,Ticker Symbol_x,Company Name_x,Price or Bid/Ask Average_x,PERMNO_y,Standard Industrial Classification Code_y,Ticker Symbol_y,Company Name_y,Price or Bid/Ask Average_y,Excess Return on the Market,Small-Minus-Big Return,High-Minus-Low Return,Momentum Factor,DATE,USRECQM
index,,,,,,,,,,,,,,,,,
1973-09-28 00:00:00+00:00,269,34367,2095,FARM,FARMER BROTHERS CO,-10.375,19764,2082,ABW,ARMADA CORP,-5.625000,0.0475,0.0293,0.0224,-0.0703,1973-10-01,1.0
1973-09-28 00:00:00+00:00,270,34367,2095,FARM,FARMER BROTHERS CO,-10.375,23106,2082,CNB,CANADIAN BREWERIES LTD,4.500000,0.0475,0.0293,0.0224,-0.0703,1973-10-01,1.0
1973-09-28 00:00:00+00:00,271,34367,2095,FARM,FARMER BROTHERS CO,-10.375,23683,2082,FAL,FALSTAFF BREWING CORP,3.250000,0.0475,0.0293,0.0224,-0.0703,1973-10-01,1.0
1973-09-28 00:00:00+00:00,272,34367,2095,FARM,FARMER BROTHERS CO,-10.375,38608,2082,GENBB,GENESEE BREWING INC,-9.625000,0.0475,0.0293,0.0224,-0.0703,1973-10-01,1.0
1973-09-28 00:00:00+00:00,273,34367,2095,FARM,FARMER BROTHERS CO,-10.375,42323,2082,RGLD,RHEINGOLD CORP,-13.125000,0.0475,0.0293,0.0224,-0.0703,1973-10-01,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-29 00:00:00+00:00,17516,90679,2095,JVA,COFFEE HOLDING CO INC,2.810,29938,2085,BF,BROWN FORMAN CORP,59.349998,0.0558,0.0245,-0.0480,0.0067,2020-04-01,1.0
2020-05-29 00:00:00+00:00,17517,90679,2095,JVA,COFFEE HOLDING CO INC,2.810,29946,2085,BF,BROWN FORMAN CORP,65.930000,0.0558,0.0245,-0.0480,0.0067,2020-04-01,1.0
2020-05-29 00:00:00+00:00,17518,90679,2095,JVA,COFFEE HOLDING CO INC,2.810,64899,2084,STZ,CONSTELLATION BRANDS INC,-172.785004,0.0558,0.0245,-0.0480,0.0067,2020-04-01,1.0


In [208]:
main_df.drop(columns = ["DATE"], inplace = True)
# main_df.to_csv("merged_final_recession.csv")